In [1]:
import requests
import pprint
import time
from bs4 import BeautifulSoup as bs
import csv
time_sleep_sec = 8
main_list = []

In [2]:
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:105.0) Gecko/20100101 Firefox/105.0'}
url = "https://hh.ru/search/vacancy"
#?no_magic=true&L_save_area=true&text=sql&area=113&experience=doesNotMatter&order_by=relevance&search_period=30&items_on_page=20

In [5]:
i = 1
main_list = []
search_text = ""
page_num = 0
mydict = {}
mystr = ""
substr = ""
mydigit = 0
mylist = []
is_ok = True
print("По умолчанию, задержка между парсингом страниц - 8 секунд\nЭто значение можно изменить в первом блоке, переменная •time_sleep_sec•")

while i < 2:
    mystr = input("Укажите число страниц с которых будет собираться собираться информация - ")
    try:
        page_num = str(mystr)
        page_num = int(page_num)
        if page_num < 1:
            print("Число должно быть положительным")
            continue
        if page_num > 50:
            print("Для учебного проекта число странмц более 50 многовато")
            continue
    except:
        print("Нужно целое число")
        pass
    else:
        break

search_text = input("Укажите специальность для поиска,\nПишите внимательно, например •sql• или •python• - ")
print()
print("Если во всех страницах, указанных по лимиту будет информация,\nПарсинг примерно займет - ", str(round((page_num * time_sleep_sec)/60)), " мин.")

print("Page parsing")
for i in range(page_num):
    
    
    param = {
    'no_magic': 'true',
    'L_save_area':'true',
    'text': search_text,
    'area': '113',
    'experience': 'doesNotMatter',
    'order_by': 'relevance',
    'search_period': '30',
    'items_on_page': '20',
    'page': i
    }

    answer = requests.get(url=url, params=param, headers=header)
    soup = bs(answer.text, 'html.parser')
    
    if i == 0:
        mystr = soup.find('h1', {'class': 'bloko-header-section-3'})
        substr = mystr.text
        mystr = ""
        for j in range(len(substr)):
            if substr[j] in "0123456789":
                mystr = mystr + substr[j]
        
        print("Всего искомых вакансий - " + mystr)
        mydigit = page_num * 20
        
        if mydigit > int(mystr):
            if int(mystr) % 20 == 0:
                print("Будет обработано - " + str(int(mystr) / 20) + " страниц,\nвместо изначальных - " + str(page_num))
                page_num = int(mystr) / 20
            else:
                print("Будет обработано - " + str(round(int(mystr) / 20) + 1) + " страниц,\nвместо изначальных - " + str(page_num))
                round((int(mystr) / 20) + 1)
    
    vacancy_employer_list = []
    mylist = soup.find_all('div', {'class': 'vacancy-serp-item-body__main-info'})
    vacancy_employer_list = soup.find_all('a', {'class': 'bloko-link bloko-link_kind-tertiary'})
    
    
    vacancy_name = ""
    vacancy_employer = ""
    vacancy_url = ""
    vacancy_salary_min = 0
    vacancy_salary_max = 0
    vacancy_salary_currency = ""
    vacancy_source = "hh.ru"
    mystr = ""
    substr2 = ""
    sublist = []
    
    
    for j in range(len(mylist)):
        mydict = {}
        mydigit = 0
        mystr = ""
        substr = ""
        substr2 = ""
        tag = mylist[j]
        vacancy_name = (tag.find('a', {'class': 'serp-item__title'}).text)
        vacancy_url =(tag.find('a', {'class': 'serp-item__title'})['href'])
        
        
        if tag.find('span', {'class': 'bloko-header-section-3'}) == None:
            vacancy_salary_min = -1
            vacancy_salary_max = -1
            vacancy_salary_currency = "none"
            
        else:
            mystr = tag.find('span', {'class': 'bloko-header-section-3'}).text
            substr2 = mystr[-4:]
            vacancy_salary_currency = substr2.strip()
            
            if mystr[0] == "о" or mystr[1] == "о":
                                
                for k in range(len(mystr)):
                    if mystr[k] in "0123456789":
                        substr = substr + mystr[k]
                
                vacancy_salary_min = int(substr)
                vacancy_salary_max = -1
                
            elif mystr[0] == "д" or mystr[1] == "д":
                               
                for k in range(len(mystr)):
                    if mystr[k] in "0123456789":
                        substr = substr + mystr[k]
                
                vacancy_salary_min = -1
                vacancy_salary_max = int(substr)
                
                
            elif mystr[0] in "123456789" or mystr[1] in "123456789":
                substr =""
                substr2 = ""
                mydigit = int((len(mystr)-4)/2)
                substr2 = mystr[:mydigit]
                                
                for k in range(len(substr2)):
                    if substr2[k] in "0123456789":
                        substr = substr + substr2[k]
                        
                vacancy_salary_min = int(substr)
                substr = ""    
                substr2 = mystr[mydigit-1:]
                                
                for k in range(len(substr2)):
                    if substr2[k] in "0123456789":
                        substr = substr + substr2[k]
                
                vacancy_salary_max = int(substr)
                
            else:
                print("Ошибка обработчика ЗП")
                print(mystr)
                pprint.pprint(mystr)
                is_ok = False
                break
        
        mydict["vacancy_name"] = vacancy_name
        mytag = vacancy_employer_list[j]
        mydict["vacancy_employer"] = mytag.text
        mydict["vacancy_url"] = vacancy_url
        mydict["vacancy_salary_min"] = vacancy_salary_min
        mydict["vacancy_salary_max"] = vacancy_salary_max
        mydict["vacancy_salary_currency"] = vacancy_salary_currency
        mydict["vacancy_source"] = vacancy_source
        main_list.append(mydict)
        
    if i + 1 == page_num:
        break
    if is_ok == False:
        break
        
    time.sleep(time_sleep_sec)
    print("Next page parsing")

По умолчанию, задержка между парсингом страниц - 8 секунд
Это значение можно изменить в первом блоке, переменная •time_sleep_sec•
Укажите число страниц с которых будет собираться собираться информация - 5
Укажите специальность для поиска,
Пишите внимательно, например •sql• или •python• - sql

Если во всех страницах, указанных по лимиту будет информация,
Парсинг примерно займет -  1  мин.
Page parsing
Всего искомых вакансий - 15690
Next page parsing
Next page parsing
Next page parsing
Next page parsing


In [8]:
with open("vacancies.csv", mode="w", encoding='utf-8') as write_file:
    file_writer = csv.writer(write_file, delimiter = ";", lineterminator="\r")
    file_writer.writerow(["vacancy_name", "vacancy_employer", "vacancy_url", "vacancy_salary_min", "vacancy_salary_max", "vacancy_salary_currency", "vacancy_source"])
    for i in range(len(main_list)):
        
        file_writer.writerow([main_list[i]["vacancy_name"], main_list[i]["vacancy_employer"], main_list[i]["vacancy_url"], main_list[i]["vacancy_salary_min"], main_list[i]["vacancy_salary_max"], main_list[i]["vacancy_salary_currency"], main_list[i]["vacancy_source"] ])
print("Done")

Done
